# Dacon 2021 농산물 가격 예측

## Library Import & Settings

###*  Light GBM
: Gradient Boosting 프레임워크로 Tree 기반 학습 알고리즘, Light GBM은 Tree가 수평적으로 확장되는 다른 알고리즘과 달리 Tree가 수직적으로 확장됨. 즉 최대 손실 값(max delta loss)를 가진 leaf를 선택하는 leaf-wise 방식을 사용. 동일한 leaf를 확장할 때, leaf-wise 알고리즘은 level-wise 알고리즘보다 더 많은 loss를 줄일 수 있음. 

이러한 장점과 달리 10,000 건 이하의 데이터셋을 다루는 경우 과적합 문제 발생하기 쉽다는 단점 존재.

###*  tqdm
: 어떤 작업을 수행중일 때, 진행상황을 알 수 있는 진행률 프로세스 바 라이브러리

###* warnings 
: 경고 메세지를 출력하고 걸러내는 기능 제공

In [11]:
import pandas as pd
import numpy as np
import lightgbm
from tqdm import tqdm
import warnings

In [12]:
# 경고 끄기
pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')        # 경고 메세지 무시
# warnings.filterwarnings(action='default')     # 경고 메세지 재활성화 시

## 전처리

### lag_feature 추가 및 기타 전처리
: lag feature란 이전 달의 정보를 추가하는 것으로 특정 품목/품종 별로 가격과 거래량이 어떻게 변했는지 column을 추가하는 것이다.

In [13]:
 # 품목/품종별로 lag_feature와 예측 대상 열을 추가한 새로운 데이터셋 생성하는 함수 정의
def preprocessing(temp_df, pum, len_lag) :
    # p_lag, q_lag 추가 (p는 가격, q는 거래량 의미)
    for lag in range(1,len_lag+1) :
      temp_df[f'p_lag_{lag}'] = -1  # p_lag_1부터 p_lag_28까지 -1로 초기화
      temp_df[f'q_lag_{lag}'] = -1  # q_lag_1부터 q_lag_28까지 -1로 초기화
      # lag가 1일때 1부터 마지막 열의 수까지
      for index in range(lag, len(temp_df)) :
        temp_df.loc[index, f'p_lag_{lag}'] = temp_df[f'{pum}_가격(원/kg)'][index-lag] # 1일전, 2일전, ... 가격을 feature로 추가(원 데이터셋의 가격 열에서 index-lag번째 행을 p_lag_lag 열의 index번째 행에 저장)
                                                                                      # temp_df.loc[1, p_lag_1] = temp_df[pum_가격(원/kg)][0]
        temp_df.loc[index, f'q_lag_{lag}'] = temp_df[f'{pum}_거래량(kg)'][index-lag]  # 1일전, 2일전, ... 거래량을 feature로 추가(원 데이터셋의 거래량 열에서 index-lag번째 행을 q_lag_lag 열의 index번째 행에 저장)
                                                                                      # temp_df.loc[1, q_lag_1] = temp_df[pum_거래량(kg)][0]

    # month 추가
    temp_df['date'] = pd.to_datetime(temp_df['date']) # datatime 형식으로 자료형 변환
    temp_df['month'] = temp_df['date'].dt.month       # date 열에서 월만 추출하여 month 열에 저장

    # 예측 대상(1w,2w,4w) 추가
    for week in ['1_week','2_week','4_week'] :
      temp_df[week] = 0     # 1w, 2w, 4w 열 초기화
      n_week = int(week[0]) # week의 첫번째 문자(1, 2, 4)를 정수로 변환 후 n_week 변수에 저장
      # index+7*n_week째 열의 가격을 week 열의 index 행에 저장
      for index in range(len(temp_df)) :
        try : temp_df[week][index] = temp_df[f'{pum}_가격(원/kg)'][index+7*n_week]
        except : continue # max(index) > index+7*n_week일 경우 초기값 그대로 0으로 저장

    # 불필요한 column 제거(date, 거래량, 가격 열 제거)
    temp_df = temp_df.drop(['date',f'{pum}_거래량(kg)',f'{pum}_가격(원/kg)'], axis=1)
    
    return temp_df

In [14]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [15]:
from google.colab import files
myfile = files.upload()

Saving train.csv to train (1).csv


In [16]:
train = pd.read_csv('train.csv')
train.head()

,date,요일,배추_거래량(kg),배추_가격(원/kg),무_거래량(kg),무_가격(원/kg),양파_거래량(kg),양파_가격(원/kg),건고추_거래량(kg),건고추_가격(원/kg),...,청상추_거래량(kg),청상추_가격(원/kg),백다다기_거래량(kg),백다다기_가격(원/kg),애호박_거래량(kg),애호박_가격(원/kg),캠벨얼리_거래량(kg),캠벨얼리_가격(원/kg),샤인마스캇_거래량(kg),샤인마스캇_가격(원/kg)
0,2016-01-01,금요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-01-02,토요일,80860.0,329.0,80272.0,360.0,122787.5,1281.0,3.0,11000.0,...,5125.0,9235.0,434.0,2109.0,19159.0,2414.0,880.0,2014.0,0.0,0.0
2,2016-01-03,일요일,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-01-04,월요일,1422742.5,478.0,1699653.7,382.0,2315079.0,1235.0,699.0,4464.0,...,38525.5,7631.0,500702.0,2046.0,620539.0,2018.0,2703.8,3885.0,0.0,0.0
4,2016-01-05,화요일,1167241.0,442.0,1423482.3,422.0,2092960.1,1213.0,1112.6,4342.0,...,32615.0,6926.0,147638.0,2268.0,231958.0,2178.0,8810.0,2853.0,0.0,0.0


In [17]:
# preprocessing 함수 예시
pum = '배추'                                                        # 품목을 배추로 설정
temp_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']] # train 데이터에서 날짜, 배추 거래량, 배추 가격 열만 추출
preprocessing(temp_df, pum, len_lag=28).tail(30)                    # 품목과 품목에 해당하는 데이터만 따로 모아서 전처리

,p_lag_1,q_lag_1,p_lag_2,q_lag_2,p_lag_3,q_lag_3,p_lag_4,q_lag_4,p_lag_5,q_lag_5,...,p_lag_26,q_lag_26,p_lag_27,q_lag_27,p_lag_28,q_lag_28,month,1_week,2_week,4_week
1703,1476,760499.0,1564,763266.0,1561,1020033.2,1373,1124756.3,1416,1119462.3,...,944,1064332.8,991,1338864.6,0,0.0,8,0,0,3066
1704,0,0.0,1476,760499.0,1564,763266.0,1561,1020033.2,1373,1124756.3,...,1011,1144604.6,944,1064332.8,991,1338864.6,8,1614,2042,1867
1705,1133,1441152.8,0,0.0,1476,760499.0,1564,763266.0,1561,1020033.2,...,939,1128530.4,1011,1144604.6,944,1064332.8,9,1994,2017,0
1706,1093,1279591.6,1133,1441152.8,0,0.0,1476,760499.0,1564,763266.0,...,1105,889162.4,939,1128530.4,1011,1144604.6,9,1585,1939,0
1707,1150,1144746.8,1093,1279591.6,1133,1441152.8,0,0.0,1476,760499.0,...,1060,997736.2,1105,889162.4,939,1128530.4,9,1542,1983,0
1708,1445,895628.0,1150,1144746.8,1093,1279591.6,1133,1441152.8,0,0.0,...,0,0.0,1060,997736.2,1105,889162.4,9,1576,1839,0
1709,1358,698187.5,1445,895628.0,1150,1144746.8,1093,1279591.6,1133,1441152.8,...,1012,1306260.4,0,0.0,1060,997736.2,9,1748,1812,0
1710,1329,1104424.8,1358,698187.5,1445,895628.0,1150,1144746.8,1093,1279591.6,...,1127,1112981.9,1012,1306260.4,0,0.0,9,0,2925,0
1711,0,0.0,1329,1104424.8,1358,698187.5,1445,895628.0,1150,1144746.8,...,1101,824042.4,1127,1112981.9,1012,1306260.4,9,2042,1813,0
1712,1614,975020.2,0,0.0,1329,1104424.8,1358,698187.5,1445,895628.0,...,1195,1186770.0,1101,824042.4,1127,1112981.9,9,2017,1838,0


## 학습

### metrics 정의

In [18]:
# 대회에서 규정한 metrics 정의
import numpy as np
import pandas as pd

def nmae(answer_df, submission_df):
    answer = answer_df.iloc[:,1:].to_numpy()
    submission = submission_df.iloc[:,1:].to_numpy()
    target_idx = np.where(answer!=0)
    
    true = answer[target_idx]
    pred = submission[target_idx]
    
    score = np.mean(np.abs(true-pred)/true)
    
    return score

def at_nmae(answer_df, submission_df):
    week_1_answer = answer_df.iloc[0::3]
    week_2_answer = answer_df.iloc[1::3]
    week_4_answer = answer_df.iloc[2::3]
    
    idx_col_nm = answer_df.columns[0]
    week_1_submission = submission_df[submission_df[idx_col_nm].isin(week_1_answer[idx_col_nm])]
    week_2_submission = submission_df[submission_df[idx_col_nm].isin(week_2_answer[idx_col_nm])]
    week_4_submission = submission_df[submission_df[idx_col_nm].isin(week_4_answer[idx_col_nm])]
    
    score1 = nmae(week_1_answer, week_1_submission)
    score2 = nmae(week_2_answer, week_2_submission)
    score4 = nmae(week_4_answer, week_4_submission)
    
    score = (score1+score2+score4)/3
    
    return score

In [19]:
# 위 코드를 모델에 맞게 수정
def nmae(week_answer, week_submission):
    answer = week_answer.to_numpy()    # to_numpy 메서드는 pandas 객체를 numpy 배열 객체인 ndarray로 반환
    target_idx = np.where(answer!=0)   # answer array 중 조건(answer!=0)에 맞는 인덱스 반환

    true = answer[target_idx]          # 실제값은 answer array에서 추출
    pred = week_submission[target_idx] # 예측값은 submission array에서 추출

    score = np.mean(np.abs(true-pred)/true) # 실제값과 예측값의 차의 절댓값을 실제값으로 나눈 것의 평균
    
    return score


def at_nmae(pred, dataset):

    # 입력값으로 받은 dataset의 레이블만 추출
    y_true = dataset.get_label()
    week_1_answer = y_true[0::3] # 레이블의 첫번째 열부터 세칸씩 띄어서 1주차 실제값에 저장
    week_2_answer = y_true[1::3] # 레이블의 두번째 열부터 세칸씩 띄어서 2주차 실제값에 저장
    week_4_answer = y_true[2::3] # 레이블의 세번째 열부터 세칸씩 띄어서 4주차 실제값에 저장
    
    week_1_submission = pred[0::3] # submission 파일의 첫번째 열부터 세칸씩 띄어서 1주차 예측값에 저장
    week_2_submission = pred[1::3] # submission 파일의 두번째 열부터 세칸씩 띄어서 2주차 예측값에 저장
    week_4_submission = pred[2::3] # submission 파일의 세번째 열부터 세칸씩 띄어서 4주차 예측값에 저장 
    
    # 위에서 정의한 함수 사용하여 주차별 nmae 계산
    score1 = nmae(week_1_answer, week_1_submission)
    score2 = nmae(week_2_answer, week_2_submission)
    score4 = nmae(week_4_answer, week_4_submission)
    
    score = (score1+score2+score4)/3 # 세 점수의 평균을 최종값으로 저장
    
    return 'score', score, False

### 학습 정의

In [20]:
# 학습 데이터와 교차 검증 데이터를 입력받는 모델 정의
def model_train(x_train, y_train, x_valid, y_valid) :
    params = {'learning_rate': 0.01,      # 0~1 사이의 값을 지정해 부스팅 스템을 반복적으로 수행할 때 업데이트 되는 학습률 값.
              'max_depth': 6,             # 트리의 최대 깊이 (모델의 과적합->max_depth 줄이기)
              'boosting': 'gbdt',         # 실행하고자 하는 알고리즘 타입 정의(gdbt: traditional gradient decision tree)
              'objective': 'regression',  # 최솟값을 가져야 할 손실함수 정의
              'is_training_metric': True, # 훈련 데이터셋에 대해 metrics 결과를 출력하고 싶을 때 True
              'num_leaves': 100,          # 하나의 트리가 가질 수 있는 최대 리프 수(num_leaves가 클수록 과적합 발생 가능성 올라감.)
              'feature_fraction': 0.8,    # Light GBM이 Tree를 만들 때 매번 각각의 iteration(반복)에서 파라미터 중에서 80%를 랜덤하게 선택
              'bagging_fraction': 0.8,    # 매번 iteration을 돌 때 사용되는 데이터의 80%를 선택하여 트레이닝 속도를 높이고 과적합을 방지
              'bagging_freq': 5,          # 5번째 반복마다 다음 5번의 반복 때 사용할 80%의 데이터를 무작위로 선택하여 bagging 실행 (0은 배깅을 사용하지 않음을 의미)
              'seed':42,                  # 매번 결과가 똑같이 나오게 하기 위해 seed 고정
              'num_threads':8             # LIghtGBM을 위한 코어의 수를 8로 설정
             }

    model = lightgbm.train(params, 
                   train_set = lightgbm.Dataset(data = x_train, label = y_train),
                   num_boost_round = 10000,                                       # boosting을 얼마나 실행할지 지정, early_stopping_round가 지정되어 있으면 더 이상 진전이 없을 경우 알아서 stop
                   valid_sets = lightgbm.Dataset(data = x_valid, label = y_valid), 
                   init_model = None,                                             # 모델 파일 경로 지정
                   early_stopping_rounds = 100,                                   # 100개의 라운드 안에 Validation 셋에서 metrics가 더 이상 향상되지 않으면 학습을 정지
                   feval = at_nmae,                                               # LightGBM에 custon metric을 적용하고 싶을 때
                   verbose_eval = False                                           # 학습 결과의 표시 빈도 지정
                    )
    
    return model

### 품목 및 품종별 모델 학습

In [21]:
# 대상 품목
unique_pum = [
    '배추', '무', '양파', '건고추','마늘',
    '대파', '얼갈이배추', '양배추', '깻잎',
    '시금치', '미나리', '당근',
    '파프리카', '새송이', '팽이버섯', '토마토',
]

# 대상 품종
unique_kind = [
    '청상추', '백다다기', '애호박', '캠벨얼리', '샤인마스캇'
]

In [22]:
#모델 훈련
model_dict = {} # 품목/품종 별, 주차 별 예측 결과 저장할 빈 딕셔너리 생성
split = 28      # validation

# 품목, 품종별로 특징이 다를 것이기에 그 특징을 살려 학습할 수 있는 개별 모델을 만드는 방법
for pum in tqdm(unique_pum + unique_kind):

    # 품목 품종별 전처리
    temp_df = train[['date',f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']] # 원데이터에서 필요한 열만 추출
    temp_df = preprocessing(temp_df, pum, len_lag=28)                   # 원데이터 이용하여 lag feature까지 포함된 새로운 데이터셋 전처리
    
    # 주차별(1,2,4w) 학습
    for week_num in [1,2,4] :
        x = temp_df[temp_df[f'{week_num}_week']>0].iloc[:,:-3]          # {week_num}_week 열에서 값이 0 이상인 행만 추출 후 마지막 열 세개만 빼고 x에 저장
        y = temp_df[temp_df[f'{week_num}_week']>0][f'{week_num}_week']  # {week_num}_week 열에서 값이 0 이상인 행만 추출 후 y에 저장
        
        #train, test split
        x_train = x[:-split]
        y_train = y[:-split]
        x_valid = x[-split:]
        y_valid = y[-split:]
        
        model_dict[f'{pum}_model_{week_num}'] = model_train(x_train, y_train, x_valid, y_valid) # 키는 {pum}_model_{week_num}, 값은 모델의 훈련 결과

100%|██████████| 21/21 [15:11<00:00, 43.41s/it]


*  21은 품종과 품목의 총 개수 의미

### 추론

In [23]:
myfile = files.upload()

Saving test_2020-09-29.csv to test_2020-09-29.csv
Saving test_2020-09-30.csv to test_2020-09-30.csv
Saving test_2020-10-01.csv to test_2020-10-01.csv
Saving test_2020-10-02.csv to test_2020-10-02.csv
Saving test_2020-10-03.csv to test_2020-10-03.csv
Saving test_2020-10-04.csv to test_2020-10-04.csv
Saving test_2020-10-05.csv to test_2020-10-05.csv
Saving test_2020-10-06.csv to test_2020-10-06.csv
Saving test_2020-10-07.csv to test_2020-10-07.csv
Saving test_2020-10-08.csv to test_2020-10-08.csv
Saving test_2020-10-09.csv to test_2020-10-09.csv
Saving test_2020-10-10.csv to test_2020-10-10.csv
Saving test_2020-10-11.csv to test_2020-10-11.csv
Saving test_2020-10-12.csv to test_2020-10-12.csv
Saving test_2020-10-13.csv to test_2020-10-13.csv
Saving test_2020-10-14.csv to test_2020-10-14.csv
Saving test_2020-10-15.csv to test_2020-10-15.csv
Saving test_2020-10-16.csv to test_2020-10-16.csv
Saving test_2020-10-17.csv to test_2020-10-17.csv
Saving test_2020-10-18.csv to test_2020-10-18.csv


In [24]:
myfile = files.upload()

Saving sample_submission.csv to sample_submission.csv


In [25]:
submission = pd.read_csv('sample_submission.csv')
public_date_list = submission[submission['예측대상일자'].str.contains('2020')]['예측대상일자'].str.split('+').str[0].unique()
# 예측대상일자 열 중 2020이 포함된 행만 추출 후 +를 기준으로 두개의 문자열로 쪼갠 후 첫번째 문자열만 선택하여 고유한 값만 리스트로 저장
# ['2020-09-29', ...] -> 총 38개의 일자

for date in tqdm(public_date_list) :
    test = pd.read_csv(f'test_{date}.csv') # 테스트 데이터 셋 불러오기(날짜별로 한개의 파일, 품목/품종별 가격과 거래량 데이터만 존재)
    for pum in unique_pum + unique_kind:   # 품목/품종 마다 각각 추론
        # 예측기준일에 대해 전처리
        temp_test = pd.DataFrame([{'date' : date}])                                      # 예측기준일로만 이루어진 열만 가지고 있는 데이터프레임 저장
        alldata = pd.concat([train, test, temp_test], sort=False).reset_index(drop=True) # train, test, 예측기준일 데이터 모두 하나의 데이터프레임으로 합치기
                                                                                         # reset_index(drop=True)는 기존 인덱스를 버리고 재배열   
        alldata = alldata[['date', f'{pum}_거래량(kg)', f'{pum}_가격(원/kg)']].fillna(0) # 날짜, 품목/품종에 해당하는 거래량, 가격 열만 선택하여 결측치는 0으로 채우기
        alldata = alldata.iloc[-28:].reset_index(drop=True)                              # 위에서 저장한 데이터셋 중 마지막 28개의 행만 선택하고 인덱스 재배열하여 저장
        alldata = preprocessing(alldata, pum, len_lag=28)                                # 함수 이용하여 전처리
        temp_test1 = alldata.iloc[-1].astype(float)                                      # 전처리 완료한 데이터셋의 마지막 행만 저장
        
        # 개별 모델을 활용하여 1,2,4주 후 가격 예측
        for week_num in [1,2,4] :
            temp_model = model_dict[f'{pum}_model_{week_num}']                  # 훈련된 개별 모델(품종/품목별, 주차별로 훈련한 모델)을 temp_model에 저장
            result = temp_model.predict(temp_test1)                             # 훈련한 모델로 시계열 데이터를 입력하여 예측, 예측 결과 result 객체에 저장
            condition = (submission['예측대상일자']==f'{date}+{week_num}week')  # submission 파일의 예측대상일자 열의 문자열과 {date}+{week_num}week가 같을 경우를 조건에 저장
            idx = submission[condition].index                                   # 위 조건에 해당하는 열의 인덱스를 idx에 저장
            submission.loc[idx, f'{pum}_가격(원/kg)'] = result[0]               # idx 행, pum_가격(원/kg) 열에 예측 결과 저장
                                                                                # array 형태로 되어 있는 result에서 값만 저장하기 위해 [0] 사용

100%|██████████| 38/38 [03:22<00:00,  5.32s/it]


In [26]:
submission.to_csv('baseline2_0920.csv',index=False)

In [27]:
pwd

'/content'